### Main code

Run the code for the full run

In [ ]:
# Normal training (without transfer learning)
import utils_convertion
import json

import utils_models
import utils_support

import random
import numpy as np
import torch
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Create coco and yolo formats
utils_convertion.process_all_subdirs('datasets/bird-detection-farm')
utils_convertion.process_all_subdirs('datasets/scarecrow_dataset')

# # Create yaml file for data configs - If these functions don't work you will have to manually adjust the path in the yaml files
utils_support.generate_yaml('bird-detection-farm', 'bird.yaml')
utils_support.generate_yaml('scarecrow_dataset', 'scarecrow.yaml')

# MURDER ALL THE CACHES (can lead to corrupted images otherwise)
utils_support.delete_cache()

# Initialize yolo model
trainer = utils_models.YOLOModel()
trainer.train(data_yaml='bird.yaml', epochs = 1, imgsz=640, cls= 0.8, iou=0.3, conf=0.1, batch_size=32)   # iou setting influences NMS. cls influences class importance (may be unnecessary)

In [ ]:
# Transfer training
import utils_convertion
import json
import utils_support

import random
import numpy as np
import torch
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Create coco and yolo formats
utils_convertion.process_all_subdirs('datasets/bird-detection-farm')
utils_convertion.process_all_subdirs('datasets/scarecrow_dataset')

# # Create yaml file for data configs - If these functions don't work you will have to manually adjust the path in the yaml files
utils_support.generate_yaml('bird-detection-farm', 'bird.yaml')
utils_support.generate_yaml('scarecrow_dataset', 'scarecrow.yaml')

# MURDER ALL THE CACHES (can lead to corrupted images otherwise)
utils_support.delete_cache()

# Initialize yolo model
from utils_models2 import YOLOModel

trainer = YOLOModel(model_path='models/pretrained/yolo11n.pt', device='cpu')

# Phase 1: Train on bird.yaml
# Phase 2: Fine-tune on scarecrow.yaml using best weights from Phase 1
trainer.transfer_training(
    additionaldata_yaml='bird.yaml',
    originaldata_yaml='scarecrow.yaml',
    intermediate_weights='models/phase1_bird.pt',
    epochs=1,
    imgsz=640,
    batch_size=32,
    cls=0.1,
    iou=0.3,
    conf=0.1,
    debug_mode=False
)

In [ ]:
# Grid search 
from utils_models2 import YOLOModel

# Initialize model 
trainer = YOLOModel(model_path='models/pretrained/yolo11n.pt', device='cpu') # alter cpu/gpu

# Run grid search using param_grid.json
trainer.grid_search(
    data_yaml='bird.yaml',
    epochs=2,                  # fewer epochs for faster testing
    result_file='grid_results.json'  # the output file name
)

with open('grid_results.json', 'r') as f:
    results = json.load(f)

best = max(results, key=lambda r: r['mAP_50'])
print("Best combo:", best["params"])
print("Best mAP@50:", best["mAP_50"])


### Converters

In [ ]:
from PIL import Image
import os

def validate_images(image_dir):
    # Iterate through all image files in the directory
    for image_filename in os.listdir(image_dir):
        image_path = os.path.join(image_dir, image_filename)
        try:
            # Try to open the image file
            img = Image.open(image_path)
            img.verify()  # Verify that it is a valid image
            print(f"Valid image: {image_path}")
        except (IOError, SyntaxError) as e:
            print(f"Corrupt image: {image_path} - Error: {e}")

# Example usage:
image_dir = 'datasets/scarecrow_dataset/train/images'
validate_images(image_dir)

### Model loops

In [ ]:
import utils_models
import utils_support

# Create yaml
#support_utils.generate_yaml(folder_name='bird-detection-farm',yaml_filename='birdfarm.yaml')

trainer = utils_models.YOLOModel()

trainer.train(data_yaml='data.yaml',epochs = 10, imgsz=640, batch_size=32, debug_mode=True)

In [ ]:
import utils_support

utils_support.visualize_yolo_annotations('datasets/scarecrow_dataset/test/images/test_original_2.png', 'datasets/scarecrow_dataset/test/labels/test_original_2.txt', class_names=['Bird'])

In [ ]:
import utils_models

trainer = utils_models.YOLOModel()

trainer.predict('datasets/scarecrow_dataset/test/images/test_original_2.png')

In [ ]:
import utils_support

utils_support.generate_yaml(yaml_filename='test.yaml')

In [ ]:
import utils_support

utils_support.delete_cache()